## Additional QC for sg659

The glider data has been processed by the base station on 2021/09/24. The software on the base stations already included some QC procedures documented [here](https://gliderfs2.coas.oregonstate.edu/sgliderweb/Seaglider_Quality_Control_Manual.html).

Here we do some:
- additional QC on the data, using the methods documented at https://glidertools.readthedocs.io/en/latest/quality_control.html
-  Calibration adjustments to remove bias, and match the observations between the two gliders. 
- Sorting and binning the data to bring it to a cleaner and easier to analyze form.

*Note that the out from this notebook is stored in this repo, and used for all the analysis for this paper. So most users might not need to run this notebook.*

In [1]:
import numpy as np
import xarray as xr

import glidertools as gt
from cmocean import cm as cmo

import gsw
import extra_funcs as ef 

import matplotlib.pyplot as plt

#### Open the dataset

In [2]:
# Edit this to be the path where the nc files downloaded from the location 
# in the read me are stored. 
data_dir = '/Users/dhruvbalwada/OneDrive/sogos_data'
nc_files_659 = data_dir + '/data/raw/gliders/SOGOS_Apr19/sg659/*.nc'

In [3]:
var_names = [
    'ctd_depth',
    'ctd_pressure',
    'salinity',
    'temperature'
]
# There are more variables measured by the gliders, but in this study we only 
# use these. 

In [4]:
ds_dict = gt.load.seaglider_basestation_netCDFs(
    nc_files_659, var_names,
    return_merged=False,
    keep_global_attrs=False
)


DIMENSION: ctd_data_point
{ctd_pressure, temperature, ctd_depth, ctd_time, latitude, salinity, longitude}


100%|██████████| 492/492 [00:02<00:00, 168.73it/s]


#### Remove non-needed dives, do additive correction, add attributes

In [5]:
ctd_659 = ds_dict['ctd_data_point']
# throw out data points after we stopped sampling the region for science
# and the gliders were being turned on just so they could get back to be picked by a ship. 
ctd_659 = ctd_659.where(ctd_659.longitude<40, drop=True) 

In [6]:
# an additive correction is made to the temperature, 
# to match to the Argo float measurements launched with the glider.
# This bias was estimated by Lily Dove at Caltech.
temp_dic = ctd_659['temperature'].attrs.copy()
ctd_659['temperature'] = ctd_659['temperature']+0.045
ctd_659['temperature'].attrs = temp_dic

In [7]:
# Load O2 data 
# (we need to use this because lily did some O2 calibration that I don't know how to do. )
# So just directly load in the processed data. 
O2_659 = xr.load_dataset('../data/O2_659.nc')
O2_659['oxygen'].attrs['standard_name'] = 'Oxygen'
O2_659['oxygen'].attrs['units'] = 'mumol/kg'

In [8]:
# Convert time axis to day format 
ctd_659['days'] = ef.datetime2ytd(ctd_659.ctd_time_dt64)
O2_659['days'] = ef.datetime2ytd(O2_659.time)

#### Do some useful thermodynamic transformations

In [9]:
# Compute the absolute salinity, conservative temperature, and potential density

SA = gsw.SA_from_SP(ctd_659.salinity, ctd_659.ctd_pressure, 
                   ctd_659.longitude, ctd_659.latitude)
CT = gsw.CT_from_t(SA, ctd_659.temperature, ctd_659.ctd_pressure)

ctd_659['SA'] = SA
ctd_659['CT'] = CT
ctd_659['sigma0'] = gsw.sigma0(SA, CT)

ctd_659['SA'].attrs['standard_name'] = 'Absolute Salinity'
ctd_659['SA'].attrs['units'] = 'g/kg'

ctd_659['CT'].attrs['standard_name'] = 'Conservative Temperature'
ctd_659['CT'].attrs['units'] = 'deg C'
ctd_659['CT'].attrs['comment'] = ctd_659['temperature'].attrs['comment']

ctd_659['sigma0'].attrs['standard_name'] = 'Potential Density Anomaly'
ctd_659['sigma0'].attrs['units'] = 'kg/m^3'
ctd_659['sigma0'].attrs['comment'] = 'Calculated using GSW'

#### Median based filter and remove some bad dives

In [10]:
filter_window = 15 # this choice made based on work above
# Do some despiking using a median filter
salt_iqr = gt.cleaning.outlier_bounds_iqr(ctd_659.SA, multiplier=1.5)
salt_base, salt_spike = gt.cleaning.despike(salt_iqr, window_size=filter_window, spike_method='median') # this is to get rid of some odd profiles

#temp_iqr = gt.cleaning.outlier_bounds_iqr(ctd_660.temperature, multiplier=2.5) # this is not a good idea for temp as it gets rid of a lot of the cold anomalies, which are real. 
temp_base, temp_spike = gt.cleaning.despike(ctd_659.CT, window_size=filter_window, spike_method='median')

O2_iqr = gt.cleaning.outlier_bounds_iqr(O2_659.oxygen, multiplier=1.5)
O2_base, O2_spike = gt.cleaning.despike(O2_iqr, window_size=filter_window, spike_method='median') # this is to get rid of some odd profiles

In [11]:
ctd_659['SA_QC'] = salt_base
ctd_659['CT_QC'] = temp_base

# We need to move O2 to ctd values
O2_659['oxygen_QC'] = O2_base
ctd_659['oxygen'] = xr.DataArray(np.interp(ctd_659.days, O2_659.days, O2_659.oxygen_QC),
                                dims= ctd_659.dims,
                                coords= ctd_659.coords).rename('oxygen')

In [12]:
# do this to get rid of some of the bad dives. 
# We will however have to manually remove a few dives that are not picked by this. 
ctd_659['oxygen_QC']  = gt.cleaning.horizontal_diff_outliers(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.oxygen, 
                                          multiplier=1.5, depth_threshold=400, mask_frac=0.1)

In [13]:
ctd_659['sigma0_QC'] = gsw.sigma0(ctd_659.SA_QC, ctd_659.CT_QC)
ctd_659['sigma0_QC'].attrs['standard_name'] = 'Potential Density Anomaly'
ctd_659['sigma0_QC'].attrs['units'] = 'kg/m^3'
ctd_659['sigma0_QC'].attrs['comment'] = 'Calculated using GSW'

### Sort the data 
Let's sort the variables based on density. "Manually finish the overturn".
We will sort the density and move the corresponding T and S with it. 

In [14]:
grp = ctd_659.groupby('dives')

ds_dives = {}
ds_dives_nonan = {}
ds_dives_sorted = {}

dim ='ctd_data_point'

for k, i in grp.groups.items():
    # extract each dive and sort in depth
    ds_dives[k] = ctd_659.isel(**{dim: i}).sortby('ctd_depth') 
    # remove nans
    ds_dives_nonan[k] = ds_dives[k].where(~np.isnan(ds_dives[k].SA_QC) & ~np.isnan(ds_dives[k].CT_QC), drop=True)
    # sort by density
    # note that this has also sorted some variables that we are not looking to be sorted.
    ds_dives_sorted[k] = ds_dives_nonan[k].sortby('sigma0_QC')
    

### Binning the data

Since every single glider profile is on a slightly different grid we grid all the data to a uniform grid in depth/pressure.

In [15]:
# bin the regular data
pres_bins = np.arange(0,1001,4)

In [16]:
# bin the sorted data 
# For this we need to do these loops since we had split the data into individual dives. 
k=2. # this number is arbitrary as we will drop this dive later, it is only used to create the right datashape
dens_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].sigma0_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)
temp_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].CT_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)
salt_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].SA_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)
O2_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].oxygen_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)

time_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].ctd_time, 
                           bins = pres_bins, verbose=False, interp_lim=0)
lat_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].latitude, 
                           bins = pres_bins, verbose=False, interp_lim=0)
lon_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].longitude, 
                           bins = pres_bins, verbose=False, interp_lim=0)


In [17]:
# loop to do it over all dives 

for k in ds_dives_sorted.keys():
    if len(ds_dives_nonan[k].ctd_data_point)>0:
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].sigma0_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        dens_sorted_gridded = xr.concat([dens_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].CT_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        temp_sorted_gridded = xr.concat([temp_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].SA_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        salt_sorted_gridded = xr.concat([salt_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].oxygen_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        O2_sorted_gridded = xr.concat([O2_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].ctd_time, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        time_sorted_gridded = xr.concat([time_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].latitude, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        lat_sorted_gridded = xr.concat([lat_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].longitude, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        lon_sorted_gridded = xr.concat([lon_sorted_gridded, temp], dim='dives')
        
# Throw away the first column since it has been added twice. 
dens_sorted_gridded = dens_sorted_gridded[:,1:]
temp_sorted_gridded = temp_sorted_gridded[:,1:]
salt_sorted_gridded = salt_sorted_gridded[:,1:]
O2_sorted_gridded   = O2_sorted_gridded[:,1:]
time_sorted_gridded = time_sorted_gridded[:,1:]
lat_sorted_gridded  = lat_sorted_gridded[:,1:]
lon_sorted_gridded  = lon_sorted_gridded[:,1:]

#### Save the data

In [18]:
ds_659_grid = xr.merge([dens_sorted_gridded.rename('sigma0'), 
                        salt_sorted_gridded.rename('SA'), 
                        temp_sorted_gridded.rename('CT'),
                        O2_sorted_gridded.rename('Oxygen'),
                        time_sorted_gridded.rename('time'), 
                        lat_sorted_gridded.rename('latitude'), 
                        lon_sorted_gridded.rename('longitude')])

In [19]:
ds_659_grid.to_netcdf('../data/sg_659_4m_binned.nc')

# The code below is what was used to check and reach the decisions that are made in the steps taken above. You might only want to run this if you want to explore how the choices were made. It is not needed to generate the data, and some cells are a repetition of the concise version above. 

**This was all done using the data processed by Geoff Shilling.**

## Prepare glider data for scientific analysis

The glider data has undergone a few things before coming to this stage. 


a) For 660 a thermal lag correction has already taken place on the basestation.   
b) 659 data, which had recovery problems, has been converted to scientific units.  
c) A correction was done to the T and S of the gliders to match them with surrounding water ($0.045$ is added to T of sg659, and $-0.18$ to the S of sg660.).  
d) A 2-sigma correction was applied to the T-S profiles. 


Further details in https://www.overleaf.com/8417818782tmwbmnqrfjry


Seaglider QC manual: https://gliderfs2.coas.oregonstate.edu/sgliderweb/Seaglider_Quality_Control_Manual.html

**One of the main goals is to make sure that the density field does not have any overturns. So smooth just enough for that and no more.**

### Some questions to think about when doing QC: 
- How do we know some data is bad? 
- One of the main concerns in the RTQCed data are overturns. How does one deal with them? Sort them?
- What degree of further smoothing should be done to the RTQCed data? Median filters, bin averages etc? 
- Is there some objective way to determine that we are in a QC sweet spot? 
- What is the role for visual inspection? 

- What to do about the 659, which did not undergo RTQC and adjustments? 
- Do we try to apply the exact same procedures as would have been done by base station? 


In [ ]:
import numpy as np
import xarray as xr

import glidertools as gt
from cmocean import cm as cmo

import gsw

import matplotlib.pyplot as plt

In [ ]:
data_dir = '/Users/dhruvbalwada/OneDrive/sogos_data'


### ... 
We want to do: 
https://glidertools.readthedocs.io/en/latest/quality_control.html

## For 659

This glider underwent all the RTQC. Here we check if any more QC is needed. 

*Note*: The dive numbers are calculated differently in Lily's files, where glider dive number was assigned by loop iteration number. This does not impact anything later, since the time and other things are correct and per data point. 

### Load data

In [ ]:
# load nc files directly
# To see the files
nc_files_659 = data_dir + '/data/raw/gliders/SOGOS_Apr19/sg659/*.nc'
#gt.load.seaglider_show_variables(nc_files_659)

In [ ]:
names = [
    'ctd_depth',
    'ctd_pressure',
    'salinity',
#    'salinity_qc',
    'temperature',
#    'temperature_qc'
]
# Don't need to get the qc flags since almost all the data point are fine for 660. 

In [ ]:
ds_dict = gt.load.seaglider_basestation_netCDFs(
    nc_files_659, names,
    return_merged=False,
    keep_global_attrs=False
)

In [ ]:
ctd_659 = ds_dict['ctd_data_point']
ctd_659 = ctd_659.where(ctd_659.longitude<40, drop=True) # throw out data points after we stopped sampling and were just trying to get back to be picked. 

In [ ]:
ctd_659

In [ ]:
# Load O2 data (we need to use this because lily did some O2 calibration that I don't know how to do. )
O2_659 = xr.load_dataset(data_dir + '/data/interim/gliders/sg659_20201010/O2_659.nc')

In [ ]:
def datetime2ytd(time):
    """" Return time in YTD format from datetime format."""
    return  (time - np.datetime64('2019-01-01'))/np.timedelta64(1, 'D')

In [ ]:
ctd_659['days'] = datetime2ytd(ctd_659.ctd_time_dt64)
O2_659['days'] = datetime2ytd(O2_659.time)

In [ ]:
SA = gsw.SA_from_SP(ctd_659.salinity, ctd_659.ctd_pressure, 
                   ctd_659.longitude, ctd_659.latitude)
CT = gsw.CT_from_t(SA, ctd_659.temperature, ctd_659.ctd_pressure)


ctd_659['SA'] = SA
ctd_659['CT'] = CT
ctd_659['sigma0'] = gsw.sigma0(SA, CT)

In [ ]:
gt.plot(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.salinity, cmap = cmo.haline, robust=True)


#### Figure out despiking choice

In [ ]:
lengths =np.array([3,5,7,10, 12, 15, 17, 20, 25, 30] )
std_salt_spikes = 0.*lengths
std_temp_spikes = 0.*lengths
std_O2_spikes   = 0.*lengths

despiked_salt = {}
despiked_temp = {}
despiked_O2   = {}

for i, lens  in enumerate(lengths):
    salt_base, salt_spike = gt.cleaning.despike(ctd_659.SA, window_size=lens, spike_method='median')    
    temp_base, temp_spike = gt.cleaning.despike(ctd_659.CT, window_size=lens, spike_method='median')    
    O2_base, O2_spike = gt.cleaning.despike(O2_659.oxygen, window_size=lens, spike_method='median')
    
    std_salt_spikes[i] = salt_spike.std().values
    std_temp_spikes[i] = temp_spike.std().values    
    std_O2_spikes[i] = O2_spike.std().values
    
    despiked_salt[lens] = {'base':salt_base, 'spikes':salt_spike}
    despiked_temp[lens] = {'base':temp_base, 'spikes':temp_spike}
    despiked_O2[lens] = {'base':O2_base, 'spikes':O2_spike}


Window size of 10-15 seems to be the sweet spot after which the spike size does not change much. 
The increase in spike size for temperature at higher values is likely a result of the filter starting to tap into the real signal. 

In [ ]:
plt.plot(lengths, std_salt_spikes, marker='o', label='SA')
plt.plot(lengths, std_temp_spikes, marker='*', label='CT')
plt.plot(lengths, std_O2_spikes, marker='o', label='Oxy')
plt.yscale('log')
plt.legend()
plt.ylabel('Spike STD')
plt.xlabel('Median window size')

### Look at the despiking 

In [ ]:
import hvplot.xarray
import panel as pn
pn.extension()

In [ ]:
def profile_filtered(var = 'SA', dive_num=300): 
    
    plot = ctd_660[var].where(ctd_660.dives==dive_num, drop=True).hvplot.line(x=var, y='ctd_depth')
    
    
    temp = ctd_660
    for i in [5, 12]:
        temp[var+'_QC'] = despiked_salt[i]['base']
    
        plot = plot*temp.where(ctd_660.dives==dive_num, drop=True).hvplot.line(x=var+'_QC', y='ctd_depth')
    
    plot.opts(width=300, height=500, invert_yaxis=True, show_legend=True)
    return plot
discrete_slider2 = pn.widgets.DiscreteSlider(name='Dive number',
                                           options= list(np.arange(1,510,0.5)),
                                           value=300)

In [ ]:
layout_salt_QC = pn.interact(profile_filtered, dive_num= discrete_slider2)
layout_salt_QC

It is a bit arbitrary, but seems like a median filter around 12 does a good job. 

### Despiking using median filters

In [ ]:
filter_window = 15 # this choice made based on work above
# Do some despiking using a median filter
salt_iqr = gt.cleaning.outlier_bounds_iqr(ctd_659.SA, multiplier=1.5)
salt_base, salt_spike = gt.cleaning.despike(salt_iqr, window_size=filter_window, spike_method='median') # this is to get rid of some odd profiles

#temp_iqr = gt.cleaning.outlier_bounds_iqr(ctd_660.temperature, multiplier=2.5) # this is not a good idea for temp as it gets rid of a lot of the cold anomalies, which are real. 
temp_base, temp_spike = gt.cleaning.despike(ctd_659.CT, window_size=filter_window, spike_method='median')

O2_iqr = gt.cleaning.outlier_bounds_iqr(O2_659.oxygen, multiplier=1.5)
O2_base, O2_spike = gt.cleaning.despike(O2_iqr, window_size=filter_window, spike_method='median') # this is to get rid of some odd profiles

In [ ]:
ctd_659['SA_QC'] = salt_base
ctd_659['CT_QC'] = temp_base

# We need to move O2 to ctd values
O2_659['oxygen_QC'] = O2_base
ctd_659['oxygen'] = xr.DataArray(np.interp(ctd_659.days, O2_659.days, O2_659.oxygen_QC),
                                dims= ctd_659.dims,
                                coords= ctd_659.coords).rename('oxygen')

In [ ]:
# do this to get rid of some of the bad dives. 
# We will however have to manually remove a few dives that are not picked by this. 
ctd_659['oxygen_QC']  = gt.cleaning.horizontal_diff_outliers(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.oxygen, 
                                          multiplier=1.5, depth_threshold=400, mask_frac=0.1)

In [ ]:
gt.plot(ctd_659.dives, ctd_659.ctd_pressure, ctd_659['oxygen_QC'], cmap = cmo.haline, robust=True)

In [ ]:
#SA = gsw.SA_from_SP(ctd_660.SA_QC, ctd_660.ctd_pressure, 
#                   ctd_660.longitude, ctd_660.latitude)
#CT = gsw.CT_from_t(SA, ctd_660.CT_QC, ctd_660.ctd_pressure)
ctd_659['sigma0_QC'] = gsw.sigma0(ctd_659.SA_QC, ctd_659.CT_QC)

In [ ]:
ctd_659['sigma0_QC'].attrs['standard_name'] = 'Potential Density Anomaly'
ctd_659['sigma0_QC'].attrs['units'] = 'kg/m^3'
ctd_659['sigma0_QC'].attrs['comment'] = 'Calculated using GSW'

In [ ]:
# hvplot setup to explore individual profiles

def profile(var='SA', dive_num=300):
    
    dive_num = float(dive_num) 
    
    plot = (ctd_660.where(ctd_660.dives==dive_num, drop=True).hvplot.line(x=var, y= 'ctd_depth')  *
            ctd_660.where(ctd_660.dives==dive_num, drop=True).hvplot.line(x = var+ '_QC', y = 'ctd_depth',
                                                                        line_width=1.) )
    plot.opts(width=300, height=500, invert_yaxis=True )
    if var=='CT':
        plot.opts(xlim=(1.2, 3.5))
    elif var=='SA': 
        plot.opts(xlim=(33.9, 35))
    return plot

discrete_slider = pn.widgets.DiscreteSlider(name='Dive number',
                                           options= list(np.arange(1,510,0.5)),
                                           value=300)

layout_salt = pn.interact(profile, dive_num= discrete_slider)
layout_temp = pn.interact(profile, var='CT', dive_num= discrete_slider)
layout_dens = pn.interact(profile, var='sigma0', dive_num= discrete_slider)

In [ ]:
pn.Column( pn.Row(layout_salt[1], layout_temp[1], layout_dens[1]), pn.Column('Profile explorer', layout_salt[0]))


The above is great. However we choose to do a little more smoothing by binning to get a vertically gridded data set on a uniform grid. This will likely also help clean some of the overturns.  

Realized that simply binning is not sufficient to get rid of all the overturns. We will use the approach of sorting the dataset based on density as well.

### Sort the data 
Let's sort the variables based on density. "Manually finish the overturn".
We will sort the density and move the corresponding T and S with it. 

In [ ]:
grp = ctd_659.groupby('dives')

In [ ]:
ds_dives = {}
ds_dives_nonan = {}
ds_dives_sorted = {}

dim ='ctd_data_point'

In [ ]:
for k, i in grp.groups.items():
    # extract each dive and sort in depth
    ds_dives[k] = ctd_659.isel(**{dim: i}).sortby('ctd_depth') 
    # remove nans
    ds_dives_nonan[k] = ds_dives[k].where(~np.isnan(ds_dives[k].SA_QC) & ~np.isnan(ds_dives[k].CT_QC), drop=True)
    # sort by density
    # note that this has also sorted some variables that we are not looking to be sorted.
    ds_dives_sorted[k] = ds_dives_nonan[k].sortby('sigma0_QC')
    

In [ ]:
ds_dives.keys()

In [ ]:
bins = [-1,0,1]
ds_dives_nonan[120].sigma0.diff(dim).plot.hist(bins = bins)
ds_dives_nonan[120].sigma0_QC.diff(dim).plot.hist(bins = bins, alpha=0.5);
ds_dives_sorted[120].sigma0_QC.diff(dim).plot.hist(bins = bins, alpha=0.5);

### Binning the data

Since every single glider profile is on a slightly different grid we grid all the data to a uniform grid.

In [ ]:
# bin the regular data
pres_bins = np.arange(0,1001,4)
dens_gridded = gt.grid_data(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.sigma0_QC, 
                           bins = pres_bins, interp_lim=0)
temp_gridded = gt.grid_data(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.CT_QC, 
                           bins = pres_bins, interp_lim=0)
salt_gridded = gt.grid_data(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.SA_QC, 
                           bins = pres_bins, interp_lim=0)
O2_gridded = gt.grid_data(ctd_659.dives, ctd_659.ctd_pressure, ctd_659.oxygen_QC, 
                           bins = pres_bins, interp_lim=0)

In [ ]:
# bin the sorted data 
# For this we need to do these loops since we had split the data into individual dives. 
k=2.
dens_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].sigma0_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)
temp_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].CT_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)
salt_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].SA_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)
O2_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].oxygen_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0)

time_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].ctd_time, 
                           bins = pres_bins, verbose=False, interp_lim=0)
lat_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].latitude, 
                           bins = pres_bins, verbose=False, interp_lim=0)
lon_sorted_gridded = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].longitude, 
                           bins = pres_bins, verbose=False, interp_lim=0)

In [ ]:
# loop to do it over all dives 

for k in ds_dives_sorted.keys():
    if len(ds_dives_nonan[k].ctd_data_point)>0:
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].sigma0_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        dens_sorted_gridded = xr.concat([dens_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].CT_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        temp_sorted_gridded = xr.concat([temp_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].SA_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        salt_sorted_gridded = xr.concat([salt_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_sorted[k].oxygen_QC, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        O2_sorted_gridded = xr.concat([O2_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].ctd_time, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        time_sorted_gridded = xr.concat([time_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].latitude, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        lat_sorted_gridded = xr.concat([lat_sorted_gridded, temp], dim='dives')
        
        temp = gt.grid_data(ds_dives_nonan[k].dives, ds_dives_nonan[k].ctd_pressure, ds_dives_nonan[k].longitude, 
                           bins = pres_bins, verbose=False, interp_lim=0);
    
        lon_sorted_gridded = xr.concat([lon_sorted_gridded, temp], dim='dives')
        
# Throw away the first column since it has been added twice. 
dens_sorted_gridded = dens_sorted_gridded[:,1:]
temp_sorted_gridded = temp_sorted_gridded[:,1:]
salt_sorted_gridded = salt_sorted_gridded[:,1:]
O2_sorted_gridded   = O2_sorted_gridded[:,1:]
time_sorted_gridded = time_sorted_gridded[:,1:]
lat_sorted_gridded  = lat_sorted_gridded[:,1:]
lon_sorted_gridded  = lon_sorted_gridded[:,1:]

In [ ]:
plt.figure(figsize=(14, 13))

plt.subplot(411)
salt_gridded.plot()
plt.gca().invert_yaxis()

plt.subplot(412)
temp_gridded.plot()
plt.gca().invert_yaxis()

plt.subplot(413)
dens_gridded.plot()
plt.gca().invert_yaxis()

plt.subplot(414)
O2_gridded.plot()
plt.gca().invert_yaxis()

In [ ]:
plt.figure(figsize=(14, 13))

plt.subplot(411)
salt_sorted_gridded.plot()
plt.gca().invert_yaxis()

plt.subplot(412)
temp_sorted_gridded.plot()
plt.gca().invert_yaxis()

plt.subplot(413)
dens_sorted_gridded.plot()
plt.gca().invert_yaxis()

plt.subplot(414)
O2_sorted_gridded.plot()
plt.gca().invert_yaxis()

In [ ]:
plt.figure(figsize=(14, 10))

plt.subplot(311)
(salt_sorted_gridded - salt_gridded).plot(vmin=-0.01)
plt.gca().invert_yaxis()

plt.subplot(312)
(temp_sorted_gridded - temp_gridded).plot(vmin=-0.1)
plt.gca().invert_yaxis()

plt.subplot(313)
(dens_sorted_gridded - dens_gridded).plot(vmin=-0.01)
plt.gca().invert_yaxis()

In [ ]:
bins = bins=np.linspace(-0.02, 0.1, 200)
dens_gridded.diff('ctd_pressure').plot.hist(bins=bins, label='Unsorted')
dens_sorted_gridded.diff('ctd_pressure').plot.hist(bins=bins, label='Sorted', alpha=0.7)
plt.legend()
plt.yscale('log')

In [ ]:
plt.figure(figsize=(14, 3.3))
dens_gridded.diff('ctd_pressure').plot(vmin=-0.04)
plt.gca().invert_yaxis()
plt.grid()

In [ ]:
plt.figure(figsize=(12,4))
ds_659_grid.Oxygen.plot(cmap=cmo.thermal)
plt.gca().invert_yaxis()

In [ ]:
# As expected sorting makes N2 purely positive. 

plt.figure(figsize=(14, 3.3))

dens_sorted_gridded.diff('ctd_pressure').plot(vmax=0.06)
plt.gca().invert_yaxis()
plt.grid()

In [ ]:
ds_659_grid = xr.merge([dens_sorted_gridded.rename('sigma0'), 
                        salt_sorted_gridded.rename('SA'), 
                        temp_sorted_gridded.rename('CT'),
                        O2_sorted_gridded.rename('Oxygen'),
                        time_sorted_gridded.rename('time'), 
                        lat_sorted_gridded.rename('latitude'), 
                        lon_sorted_gridded.rename('longitude')])